In [14]:
# importations 
import pandas as pd

### Data pre-processing

In [15]:
df = pd.read_csv("00. input/train.csv", sep=';', decimal=".")
df_test = pd.read_csv("00. input/test.csv", sep=';', decimal=".")

In [16]:
# on transorme la colonne des types en 3 colonnes dummies 

df = pd.concat([df, pd.get_dummies(df["Type"]).astype("int64")], axis=1)
df_test = pd.concat([df_test, pd.get_dummies(df_test["Type"]).astype("int64")], axis=1)

In [17]:
y = df["Machine failure"]
X = df.drop(columns=["id", "Machine failure"]).select_dtypes(include=['float64', 'int64'])
X_test = df_test.drop(columns=["id"]).select_dtypes(include=['float64', 'int64'])

In [18]:
X_test

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,H,L,M
0,302.3,311.5,1499,38.0,60,0,0,0,0,0,0,1,0
1,301.7,311.0,1713,28.8,17,0,0,0,0,0,0,1,0
2,301.3,310.4,1525,37.7,96,0,0,0,0,0,0,1,0
3,300.1,309.6,1479,47.6,5,0,0,0,0,0,0,0,1
4,303.4,312.3,1515,41.3,114,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,302.3,311.4,1484,40.4,15,0,0,0,0,0,0,1,0
90950,297.9,309.8,1542,33.8,31,0,0,0,0,0,0,1,0
90951,295.6,306.2,1501,41.4,187,0,0,0,0,0,0,1,0
90952,298.1,307.8,1534,40.3,69,0,0,0,0,0,0,1,0


### Je tente une simple random forest

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', RandomForestClassifier())
])

# je fit avec tte la donnée de train
pipeline.fit(X, y)

Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])

In [20]:
# je prédis la proba de la machine failure donc la 2 emme colonne 
y_test = pd.concat([df_test["id"], pd.DataFrame(pipeline.predict_proba(X_test), columns=["proba_0", "Machine failure"])["Machine failure"]], axis=1)

In [21]:
(y_test["Machine failure"].isna()).sum()

0

In [24]:
y_test

,id,Machine failure
0,136429,0.0
1,136430,0.0
2,136431,0.0
3,136432,0.0
4,136433,0.0
...,...,...
90949,227378,0.0
90950,227379,0.0
90951,227380,0.0
90952,227381,0.0


In [23]:
y_test.to_csv("01. output/soumission_2.csv", decimal='.', sep=',', index=False)

In [ ]:
# simple pipeline

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', RandomForestClassifier())
])

scores = cross_val_score(pipeline, X, y, cv=5, scoring='roc_auc')
print('Scores de validation croisée :', scores)
print('Précisiono moyenne :', scores.mean())

KeyboardInterrupt: 

In [ ]:
# simple pipeline

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', RandomForestClassifier())
])

# défintion de la grille des paramètres à explorer
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

# Création de l'objet GridSearchCV avec la pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')

# Recherche des hyperparamètres
grid_search.fit(X, y)


# Affichage des meilleurs hyperparamètres et du meilleur score ROC AUC
print('Meilleurs hyperparamètres :', grid_search.best_params_)
print('Meilleur score ROC AUC :', grid_search.best_score_)

KeyboardInterrupt: 

In [ ]:
# simple pipeline

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), # remplissage des valeurs manquantes par un imputer
    ('scaler', StandardScaler()), # mise à l'échelle des données
    ('classifier', RandomForestClassifier(
        # n_estimators=
        # max_depth=
        # min_samples_split=
        # min_samples_leaf=
    )
    )
])

# défintion de la grille des paramètres à explorer
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

# Création de l'objet GridSearchCV avec la pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')

# Recherche des hyperparamètres
grid_search.fit(X, y)


# Affichage des meilleurs hyperparamètres et du meilleur score ROC AUC
print('Meilleurs hyperparamètres :', grid_search.best_params_)
print('Meilleur score ROC AUC :', grid_search.best_score_)